<a href="https://colab.research.google.com/github/EmTampz/Copra_YOLOv11_v12_Colab_Notebooks/blob/main/Copra_1_manual_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ==========================================================
# 📦 ENVIRONMENT SETUP for YOLOv11 Thesis (Colab)

# Core deep learning and YOLO library
!pip install -U ultralytics

# Core data handling, visualization, and ML utilities
!pip install pandas matplotlib seaborn pillow scikit-learn

#local dashboard:  monitoring and logging
!pip install tensorboard tqdm

# Roboflow integration
!pip install roboflow

# cloud-connected research tracker
!pip -q install wandb


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 74.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 104.1 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.12.0.88
    Uninstalling opencv-python-headless-4.12.0.88:
      Successfully uninstalled opencv-python-headless-4.12.0.88
  Attempting uninstall: idna
    Found existing installation: idna 3.11
    Uninstalling idna-3.11:
      Successfully uninstalled idna-3.11


In [2]:
import torch, ultralytics, pandas as pd, matplotlib, seaborn, PIL, sklearn, tqdm, tensorboard

print(f"PyTorch version : {torch.__version__}")
print(f"CUDA available  : {torch.cuda.is_available()}")
print(f"Ultralytics ver : {ultralytics.__version__}")
ultralytics.checks()

Ultralytics 8.3.228 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 38.4/235.7 GB disk)


In [3]:
# ===============================================================
# 🧩 BLOCK 1 — SETUP, IMPORTS, AND CONFIGURATION
# ===============================================================

import os, time, json, yaml, glob, zipfile, random
from datetime import datetime
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import (
    confusion_matrix, ConfusionMatrixDisplay,
    precision_recall_fscore_support, accuracy_score,
)
import cv2
from ultralytics import YOLO
from roboflow import Roboflow
import wandb
from IPython.display import display

# ---------------- MOUNT GOOGLE DRIVE ----------------
try:
    from google.colab import drive
    drive.mount('/content/drive')
    DRIVE_OK = True
    print("✅ Google Drive mounted successfully.")
except Exception:
    DRIVE_OK = False
    print("⚠️ Drive not mounted. Using local storage.")

BASE_DIR = "/content/drive/MyDrive/yolo_copra" if DRIVE_OK else "/content/yolo_copra"

# ✅ Updated folder path for saving results
RESULTS_DIR = os.path.join(BASE_DIR, "copra_yolo11_detect_results_new")
os.makedirs(RESULTS_DIR, exist_ok=True)
print(f"📁 Results directory: {RESULTS_DIR}")

# ---------------- WANDB SETUP ----------------
os.environ["WANDB_API_KEY"] = "02a2986eed98555093c4c2920a04c631a89430cc"
!wandb login $WANDB_API_KEY --relogin
WANDB_PROJECT = "Copra-YOLOv11-Detection"

# ---------------- LOAD ROBOFLOW DATASET ----------------
rf = Roboflow(api_key="LS1Ohi2OQZ178OOqQ4zc")
project = rf.workspace("cv-opfhv").project("mor-v6-jj3ic")
dataset = project.version(1).download("yolov11")
DATA_YAML = os.path.join(dataset.location, "data.yaml")

with open(DATA_YAML, "r") as f:
    meta = yaml.safe_load(f)
class_names = [c for c in meta["names"] if c.lower() not in ["background", "null"]]
print(f"✅ Dataset loaded with {len(class_names)} classes: {class_names}")


⚠️ Drive not mounted. Using local storage.
📁 Results directory: /content/yolo_copra/copra_yolo11_detect_results_new
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to MOR-V6-1 in yolov11:: 100%|██████████| 8506/8506 [00:00<00:00, 10157.38it/s]


✅ Dataset loaded with 3 classes: ['Over-Cooked', 'Perfectly-Cooked', 'Under-Cooked']


In [4]:
# ===============================================================
# 4️⃣ CONFIGURATION
# ===============================================================
CONFIG = dict(
    epochs=300,
    batch=16,
    lr0=0.001,
    patience=100,
    optimizer="auto",
    imgsz=640,
)
MODEL_PATH = "yolo11n.pt"

In [5]:

# ---------------- UTILITIES ----------------
def latest_checkpoint(base_dir: str):
    ckpts = glob.glob(os.path.join(base_dir, "**", "*.pt"), recursive=True)
    return max(ckpts, key=os.path.getmtime) if ckpts else None

def zip_results(folder: str):
    zip_path = os.path.join(folder, f"backup_{datetime.now().strftime('%Y%m%d_%H%M%S')}.zip")
    with zipfile.ZipFile(zip_path, "w", zipfile.ZIP_DEFLATED) as z:
        for root, _, files in os.walk(folder):
            for file in files:
                fp = os.path.join(root, file)
                z.write(fp, os.path.relpath(fp, start=folder))
    return zip_path

def parse_label_file(label_path: str):
    labels = []
    if os.path.exists(label_path):
        with open(label_path) as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) >= 5:
                    cls = int(parts[0])
                    if cls < len(class_names):
                        labels.append(cls)
    return labels

def save_and_show(fig_path: str, title: str = None):
    if title: plt.title(title)
    plt.tight_layout(); plt.savefig(fig_path, bbox_inches="tight"); plt.show()

In [6]:
# ===============================================================
# 🧩 BLOCK 2 — TRAINING + VALIDATION (FIXED: SINGLE FOLDER OUTPUT)
# ===============================================================
def train_yolo_detection():
    model = YOLO(MODEL_PATH)
    print("🚀 Starting YOLOv11 Detection Training...")

    # Auto-resume detection
    existing_runs = sorted([d for d in os.listdir(RESULTS_DIR) if os.path.isdir(os.path.join(RESULTS_DIR, d))])
    resume, run_name, ckpt = False, None, None
    if existing_runs:
        latest_run_dir = os.path.join(RESULTS_DIR, existing_runs[-1])
        ckpt = latest_checkpoint(latest_run_dir)
        if ckpt:
            resume, run_name = True, os.path.basename(latest_run_dir)
            print(f"🔁 Auto-resume detected: {ckpt}")
        else:
            print("⚠️ No valid checkpoint found. Starting new training.")
    else:
        print("🆕 Starting fresh training run.")

    # W&B naming and setup
    if not run_name:
        run_name = f"det_e{CONFIG['epochs']}_b{CONFIG['batch']}_lr{CONFIG['lr0']}_{CONFIG['optimizer']}_img{CONFIG['imgsz']}_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
    run_dir = os.path.join(RESULTS_DIR, run_name)
    os.makedirs(run_dir, exist_ok=True)

    wandb.init(project=WANDB_PROJECT, name=run_name, resume="allow", config=CONFIG)
    wandb.config.update({"group": CONFIG["optimizer"]})

    # ===============================================================
    # 🏋️‍♂️ TRAINING STARTS
    # ===============================================================
    start_time = time.time()
    results = model.train(
        data=DATA_YAML,
        task="detect",
        epochs=CONFIG["epochs"],
        batch=CONFIG["batch"],
        lr0=CONFIG["lr0"],
        patience=CONFIG["patience"],
        optimizer=CONFIG["optimizer"],
        imgsz=CONFIG["imgsz"],

        # 🟩 CHANGE 1: Force YOLO to save inside the same run_dir
        project=run_dir,   # instead of RESULTS_DIR
        name="",           # prevents YOLO from creating nested folders
        exist_ok=True,     # overwrite instead of creating “_2”
        save_period=5,
        resume=False,
    )

    # 🟥 REMOVE this line — YOLO already saves in run_dir
    # run_dir = str(results.save_dir)

    print(f"🏁 Training complete! All outputs stored in: {run_dir}")

    # ===============================================================
    # 🔍 VALIDATION
    # ===============================================================
    print("🔎 Running validation...")
    val = model.val(
        project=os.path.join(run_dir, "validation"),  # 🟩 CHANGE 2: create subfolder
        name="", exist_ok=True
    )

    # 📊 Metric extraction (Ultralytics v8/11+)
    metrics = {
        "Precision": float(val.box.mp),
        "Recall": float(val.box.mr),
        "mAP@0.5": float(val.box.map50),
        "mAP@0.5:0.95": float(val.box.map),
    }
    metrics["F1"] = 2 * (metrics["Precision"] * metrics["Recall"]) / (
        metrics["Precision"] + metrics["Recall"] + 1e-6
    )

    # Display + Save
    dfm = pd.DataFrame([metrics])
    display(dfm.style.background_gradient(cmap="YlGnBu").format(precision=3))
    dfm.to_csv(os.path.join(run_dir, "summary_metrics.csv"), index=False)
    wandb.log(metrics)
    wandb.finish()

    # ===============================================================
    # ⏱️ TIMING + CHECKPOINT VALIDATION
    # ===============================================================
    elapsed = time.time() - start_time
    hours, minutes = divmod(elapsed / 60, 60)
    print(f"🕒 Training + Validation completed in {int(hours)}h {int(minutes)}m ({elapsed:.2f}s total)")
    print(f"📁 Run saved in: {run_dir}")

    # ✅ FIX 3 — Confirm weights actually exist
    weights_path = os.path.join(run_dir, "weights")
    if os.path.exists(weights_path):
        print("✅ Weights folder verified:", weights_path)
        os.system(f'ls "{weights_path}"')
    else:
        print("⚠️ Warning: No weights folder found. Drive may have unmounted or run interrupted.")

    return run_dir


In [7]:
# ===============================================================
# 🧩 BLOCK 3 — TESTING (INFERENCE)
# ===============================================================
def run_yolo_detection_test(run_dir: str):
    test_dir = os.path.join(run_dir, "test_results")
    os.makedirs(test_dir, exist_ok=True)
    best_path = os.path.join(run_dir, "weights", "best.pt")

    if not os.path.exists(best_path):
        candidates = glob.glob(os.path.join(BASE_DIR, "**", "best.pt"), recursive=True)
        if not candidates:
            raise FileNotFoundError("❌ No best.pt found!")
        best_path = max(candidates, key=os.path.getmtime)
    print(f"✅ Using checkpoint: {best_path}")

    model = YOLO(best_path)
    test_imgs = glob.glob(os.path.join(dataset.location, "test", "images", "*.jpg"))
    print(f"🧩 Found {len(test_imgs)} test images.")

    y_true, y_pred, conf_scores = [], [], []
    for img_path in test_imgs:
        res = model.predict(img_path, conf=0.25, iou=0.45, verbose=False)[0]
        gt = parse_label_file(Path(img_path).with_suffix(".txt"))
        img = cv2.imread(img_path)
        for i, box in enumerate(res.boxes.xyxy):
            cls = int(res.boxes.cls[i]); conf = float(res.boxes.conf[i])
            conf_scores.append(conf); y_pred.append(cls)
            x1, y1, x2, y2 = map(int, box)
            cv2.rectangle(img, (x1, y1), (x2, y2), (0,255,0), 2)
            cv2.putText(img, f"{class_names[cls]} {conf:.2f}", (x1, max(20, y1-10)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 2)
        cv2.imwrite(os.path.join(test_dir, f"bbox_{Path(img_path).name}"), img[:, :, ::-1])
        y_true.extend(gt)

    print(f"✅ Inference complete. Results saved in {test_dir}")
    return y_true, y_pred, conf_scores, test_dir


In [8]:
# ===============================================================
# 🧩 BLOCK 4 — VISUALIZATION (POST-TEST METRIC ANALYSIS)
# ===============================================================
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, precision_recall_fscore_support

# 🔹 Class names
class_names = ["Undercooked", "Perfectly Cooked", "Overcooked"]

def visualize_test_results(y_true, y_pred, conf_scores, test_dir):
    valid_idx = list(range(len(class_names)))
    y_true = [y for y in y_true if y in valid_idx]
    y_pred = [y for y in y_pred if y in valid_idx]

    # Compute metrics
    precision, recall, f1, support = precision_recall_fscore_support(
        y_true, y_pred, labels=valid_idx, zero_division=0
    )
    cm = confusion_matrix(y_true, y_pred, labels=valid_idx)
    conf_scores = np.array(conf_scores)
    mean_conf, std_conf = np.mean(conf_scores), np.std(conf_scores)
    ci_low, ci_high = mean_conf - std_conf, mean_conf + std_conf
    print(f"📈 Confidence Interval: {mean_conf:.3f} ± {std_conf:.3f} → [{ci_low:.3f}, {ci_high:.3f}]")

    # Export metrics
    metrics_df = pd.DataFrame({
        "Label": class_names,
        "Precision": precision,
        "Recall": recall,
        "F1-Score": f1,
        "Accuracy (%)": (cm.diagonal() / cm.sum(axis=1)) * 100
    })
    display(metrics_df.style.background_gradient(cmap="YlOrBr").format(precision=3))
    metrics_df.to_csv(os.path.join(test_dir, "test_metrics.csv"), index=False)

    # 🎯 Confusion Matrix
    plt.figure(figsize=(6,6))
    ConfusionMatrixDisplay(cm, display_labels=class_names).plot(values_format=".0f", cmap="Blues")
    plt.title("Confusion Matrix (YOLOv11)")
    plt.xlabel("Predicted Label")
    plt.ylabel("True Label")
    plt.tight_layout()
    plt.savefig(os.path.join(test_dir, "confusion_matrix_test.png"))
    plt.show()

    # 📊 Metric Graphs
    plt.figure(figsize=(8,5))
    plt.bar(class_names, metrics_df["Accuracy (%)"], color="cornflowerblue")
    plt.title("Per-Class Accuracy (YOLOv11)")
    plt.ylim(0,100); plt.grid(alpha=0.4)
    plt.tight_layout(); plt.savefig(os.path.join(test_dir,"per_class_accuracy_test.png")); plt.show()

    plt.figure(figsize=(8,5))
    plt.bar(class_names, precision, color="seagreen")
    plt.title("Per-Class Precision (YOLOv11)")
    plt.ylim(0,1); plt.grid(alpha=0.4)
    plt.tight_layout(); plt.savefig(os.path.join(test_dir,"per_class_precision_test.png")); plt.show()

    plt.figure(figsize=(8,5))
    plt.bar(class_names, recall, color="gold")
    plt.title("Per-Class Recall (YOLOv11)")
    plt.ylim(0,1); plt.grid(alpha=0.4)
    plt.tight_layout(); plt.savefig(os.path.join(test_dir,"per_class_recall_test.png")); plt.show()

    plt.figure(figsize=(8,5))
    plt.bar(class_names, f1, color="tomato")
    plt.title("Per-Class F1-Score (YOLOv11)")
    plt.ylim(0,1); plt.grid(alpha=0.4)
    plt.tight_layout(); plt.savefig(os.path.join(test_dir,"per_class_f1_test.png")); plt.show()

    # 📈 Confidence Histogram
    plt.figure(figsize=(6,4))
    plt.hist(conf_scores,bins=20,color="skyblue",edgecolor="k")
    plt.axvline(mean_conf,color="red",ls="--",label=f"μ={mean_conf:.2f}")
    plt.legend(); plt.xlabel("Confidence"); plt.ylabel("Frequency")
    plt.title("Confidence Distribution (YOLOv11)")
    plt.tight_layout(); plt.savefig(os.path.join(test_dir,"confidence_hist_test.png")); plt.show()

    # 🔵 Precision–Recall Scatter
    plt.figure(figsize=(6,6))
    plt.scatter(precision, recall, s=80, c='dodgerblue')
    for i,cls in enumerate(class_names):
        plt.text(precision[i]+0.01, recall[i], cls, fontsize=9)
    plt.xlabel("Precision"); plt.ylabel("Recall")
    plt.title("Precision–Recall Scatter (YOLOv11)")
    plt.grid(True,ls="--",alpha=0.5)
    plt.tight_layout(); plt.savefig(os.path.join(test_dir,"precision_recall_scatter_test.png")); plt.show()

    print(f"✅ Visualizations saved in: {test_dir}")


In [11]:
# ===============================================================
# 🧩 BLOCK 5A — AUTO-RESUME OR TRAIN (FINAL FIXED VERSION)
# ===============================================================
import os, glob
from ultralytics import YOLO

RESULTS_DIR = "/content/drive/MyDrive/yolo_copra/copra_yolo11_detect_results_new"

def auto_resume_training():
    """
    Resume YOLOv11 detection training if a checkpoint exists.
    If none found, automatically start a new training run using train_yolo_detection().
    """
    candidate_ckpts = [
        ckpt for ckpt in glob.glob(os.path.join(RESULTS_DIR, "**", "*.pt"), recursive=True)
        if any(x in ckpt for x in ["last.pt", "best.pt", "epoch_"])
    ]

    # ✅ Case 1: No checkpoints found → train fresh model
    if not candidate_ckpts:
        print("⚠️ No checkpoints found — starting a fresh YOLOv11 training run...")
        run_dir = train_yolo_detection()  # your function already includes validation
        print("✅ Initial training (and validation) completed successfully.")
        return run_dir

    # ✅ Case 2: Checkpoint found → resume from last/best
    ckpt_path = max(candidate_ckpts, key=os.path.getmtime)
    run_dir = os.path.dirname(os.path.dirname(ckpt_path))
    print(f"⚙️ Found checkpoint: {ckpt_path}")

    DATA_YAML = os.path.join(dataset.location, "data.yaml")
    if not os.path.exists(DATA_YAML):
        raise FileNotFoundError(f"❌ Dataset YAML not found at {DATA_YAML}")

    try:
        print(f"📂 Attempting to resume training from: {run_dir}")
        model = YOLO(ckpt_path)
        model.train(
            resume=True,
            data=DATA_YAML,
            project=RESULTS_DIR,
            name=os.path.basename(run_dir),
        )
        print("🎯 Training resumed successfully.")
        return run_dir

    except AssertionError as e:
        if "nothing to resume" in str(e):
            print("✅ Training already completed. Proceeding directly to testing phase.")
            return run_dir
        else:
            raise e


In [12]:
# ===============================================================
# 🧠 BLOCK 5B — TEST MODEL AFTER TRAINING (SYNCHRONIZED FLOW, FINAL)
# ===============================================================
import os, glob
import pandas as pd
import numpy as np
from ultralytics import YOLO
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support, accuracy_score

RESULTS_DIR = "/content/drive/MyDrive/yolo_copra/copra_yolo11_detect_results_new"

# ===============================================================
# 🔁 STEP 0 — RESUME OR LOCATE MODEL
# ===============================================================
run_dir = auto_resume_training()

if run_dir is None:
    print("🚀 No resume or prior training detected — locating latest trained model...")
    best_models = glob.glob(os.path.join(RESULTS_DIR, "**", "best.pt"), recursive=True)
    if not best_models:
        raise FileNotFoundError("❌ No best.pt found for testing phase.")
    latest_best = max(best_models, key=os.path.getmtime)
    run_dir = os.path.dirname(os.path.dirname(latest_best))
    model_path = latest_best
else:
    model_path = os.path.join(run_dir, "weights", "best.pt")
    if not os.path.exists(model_path):
        model_path = os.path.join(run_dir, "weights", "last.pt")

# ===============================================================
# 🧩 STEP 0.5 — AUTO-SYNC ACTUAL YOLO FOLDER (handles "_2" issue)
# ===============================================================
expected_weights = os.path.join(run_dir, "weights")
if not os.path.exists(expected_weights):
    print(f"⚠️ Folder mismatch: {expected_weights} not found. Searching for true run folder...")
    run_name = os.path.basename(run_dir)
    alt_candidates = glob.glob(os.path.join(RESULTS_DIR, run_name + "*"))
    if alt_candidates:
        run_dir = max(alt_candidates, key=os.path.getmtime)
        print(f"⚙️ Synced run_dir to actual YOLO folder: {run_dir}")
        model_path = os.path.join(run_dir, "weights", "best.pt")
        if not os.path.exists(model_path):
            model_path = os.path.join(run_dir, "weights", "last.pt")
    else:
        raise FileNotFoundError(f"❌ Could not find matching folder for {run_name}")

if not os.path.exists(model_path):
    raise FileNotFoundError(f"❌ Still no valid checkpoint found at {model_path}")

print(f"✅ Using model for testing: {model_path}")

# ===============================================================
# 🧪 STEP 1 — VALIDATION ON TEST SPLIT
# ===============================================================
model = YOLO(model_path)

# Ensure test_results folder exists before saving
test_dir = os.path.join(run_dir, "test_results")
os.makedirs(test_dir, exist_ok=True)
print(f"📁 Validation results will be saved under: {test_dir}")

metrics = model.val(
    save_json=True,
    save_hybrid=True,
    imgsz=640,
    project=test_dir,      # ✅ force save outputs in same folder
    name="",               # ✅ prevent nested folders
    exist_ok=True
)

print("\n📊 Validation Summary Metrics:")
for k, v in metrics.results_dict.items():
    if isinstance(v, (int, float, np.floating)):
        print(f" - {k}: {v:.4f}")
    else:
        print(f" - {k}: {v}")

# ✅ Safe save to summary CSV
metrics_df = pd.DataFrame([metrics.results_dict])
summary_path = os.path.join(test_dir, "summary_metrics.csv")
metrics_df.to_csv(summary_path, index=False)
print(f"💾 Summary metrics saved to: {summary_path}")

# ===============================================================
# 🧠 STEP 2 — INFERENCE FOR DETAILED OUTPUTS (per-image predictions)
# ===============================================================
print("\n🔍 Running inference for per-image results...")

test_images = os.path.join(dataset.location, "test", "images")
if not os.path.exists(test_images):
    raise FileNotFoundError(f"❌ Test image folder not found: {test_images}")
else:
    print(f"🧩 Found {len(glob.glob(os.path.join(test_images, '*.jpg')))} test images.")

pred_results = model.predict(
    source=test_images,
    save=True,
    project=test_dir,    # ✅ all results under test_results/
    name="predictions",
    exist_ok=True
)

y_true, y_pred, conf_scores = [], [], []
for r in pred_results:
    if r.boxes is not None:
        preds = r.boxes
        y_pred.extend(preds.cls.cpu().numpy())
        conf_scores.extend(preds.conf.cpu().numpy())

print(f"✅ Inference complete. Saved predictions to: {os.path.join(test_dir, 'predictions')}")
print(f"📂 Test results fully stored in: {run_dir}")


⚠️ No checkpoints found — starting a fresh YOLOv11 training run...
🚀 Starting YOLOv11 Detection Training...


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/yolo_copra/copra_yolo11_detect_results_new'

In [ ]:
# ===============================================================
# 🧩 BLOCK 5C — VISUALIZE TEST RESULTS (TRUE CURVES + SAFETY PRINTS)
# ===============================================================
import os, glob
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.metrics import (
    confusion_matrix, ConfusionMatrixDisplay,
    precision_recall_fscore_support, precision_recall_curve
)

# 🔹 Class names for label IDs 0, 1, 2
class_names = ["Undercooked", "Perfectly Cooked", "Overcooked"]

def load_ground_truth_labels(labels_dir):
    """Read YOLO-format .txt files and extract class IDs."""
    y_true = []
    if not os.path.exists(labels_dir):
        print(f"⚠️ Ground-truth labels directory not found: {labels_dir}")
        return y_true
    label_files = glob.glob(os.path.join(labels_dir, "*.txt"))
    if not label_files:
        print(f"⚠️ No label files found inside {labels_dir}")
        return y_true
    for lf in label_files:
        with open(lf) as f:
            for line in f:
                parts = line.strip().split()
                if parts:
                    try:
                        y_true.append(int(float(parts[0])))
                    except ValueError:
                        continue
    print(f"✅ Loaded {len(y_true)} ground-truth labels from {labels_dir}")
    return y_true


def visualize_test_results(y_true, y_pred, conf_scores, test_dir):
    """Full metric and visual report (YOLOv11)."""
    os.makedirs(test_dir, exist_ok=True)
    print(f"📂 Saving visual outputs to: {test_dir}")

    if len(y_true) == 0 or len(y_pred) == 0:
        print("⚠️ Empty y_true or y_pred — skipping visualization.")
        return

    valid_idx = list(range(len(class_names)))
    y_true = [y for y in y_true if y in valid_idx]
    y_pred = [y for y in y_pred if y in valid_idx]

    precision, recall, f1, support = precision_recall_fscore_support(
        y_true, y_pred, labels=valid_idx, zero_division=0
    )
    cm = confusion_matrix(y_true, y_pred, labels=valid_idx)
    conf_scores = np.array(conf_scores) if conf_scores is not None else np.array([])

    # 📊 Build DataFrame
    metrics_df = pd.DataFrame({
        "Label": class_names,
        "Precision": precision,
        "Recall": recall,
        "F1-Score": f1,
        "Accuracy (%)": (cm.diagonal() / cm.sum(axis=1)) * 100
    })
    print("📊 Per-class metrics:")
    print(metrics_df.round(3))
    metrics_df.to_csv(os.path.join(test_dir, "test_metrics.csv"), index=False)

    # 🟦 Confusion Matrix
    plt.figure(figsize=(4, 4))
    ConfusionMatrixDisplay(cm, display_labels=class_names).plot(values_format=".0f", cmap="Blues")
    plt.title("Confusion Matrix (YOLOv11)")
    plt.xlabel("Predicted Label")
    plt.ylabel("True Label")
    plt.tight_layout()
    plt.savefig(os.path.join(test_dir, "confusion_matrix_yolov11.png"))
    plt.show()

    # 🟩 Metric Bars
    for metric, color in zip(["Precision", "Recall", "F1-Score"], ["mediumseagreen", "gold", "tomato"]):
        plt.figure(figsize=(4, 4))
        plt.bar(class_names, metrics_df[metric], color=color)
        plt.ylim(0, 1)
        plt.ylabel(metric)
        plt.title(f"Per-Class {metric} (YOLOv11)")
        plt.grid(True, ls="--", alpha=0.4)
        plt.tight_layout()
        plt.savefig(os.path.join(test_dir, f"per_class_{metric.lower()}_yolov11.png"))
        plt.show()

    # 🟨 Accuracy
    plt.figure(figsize=(4, 4))
    plt.bar(class_names, metrics_df["Accuracy (%)"], color="cornflowerblue")
    plt.ylim(0, 100)
    plt.ylabel("Accuracy (%)")
    plt.title("Per-Class Accuracy (YOLOv11)")
    plt.grid(True, ls="--", alpha=0.4)
    plt.tight_layout()
    plt.savefig(os.path.join(test_dir, "per_class_accuracy_yolov11.png"))
    plt.show()

    # 🟦 Confidence Histogram
    if len(conf_scores) > 0:
        mean_conf, std_conf = np.mean(conf_scores), np.std(conf_scores)
        plt.figure(figsize=(4, 4))
        plt.hist(conf_scores, bins=20, color="skyblue", edgecolor="k")
        plt.axvline(mean_conf, color="red", ls="--", label=f"μ={mean_conf:.2f}")
        plt.legend()
        plt.xlabel("Confidence")
        plt.ylabel("Frequency")
        plt.title("Confidence Distribution (YOLOv11)")
        plt.grid(alpha=0.3)
        plt.tight_layout()
        plt.savefig(os.path.join(test_dir, "confidence_hist_yolov11.png"))
        plt.show()
    else:
        print("⚠️ No confidence scores found — skipping confidence histogram.")

    # 🔵 Precision–Recall Scatter
    plt.figure(figsize=(4, 4))
    plt.scatter(precision, recall, s=80, c="dodgerblue")
    for i, cls in enumerate(class_names):
        plt.text(precision[i] + 0.01, recall[i], cls, fontsize=9)
    plt.xlabel("Precision")
    plt.ylabel("Recall")
    plt.title("Precision–Recall Scatter (YOLOv11)")
    plt.grid(True, ls="--", alpha=0.5)
    plt.tight_layout()
    plt.savefig(os.path.join(test_dir, "precision_recall_scatter_yolov11.png"))
    plt.show()

    # ===============================================================
    # 🔶 ADDITIONAL CURVES (True per-class Precision, Recall, F1)
    # ===============================================================
    if len(conf_scores) > 0 and len(y_true) == len(y_pred):
        try:
            print("📈 Generating true per-class performance curves...")
            for i, cls in enumerate(class_names):
                y_true_cls = np.array([1 if t == i else 0 for t in y_true])
                y_score_cls = np.array([
                    conf_scores[j] if y_pred[j] == i else 0 for j in range(len(y_pred))
                ])
                if y_true_cls.sum() == 0:
                    print(f"⚠️ Skipping '{cls}' — no true samples in test set.")
                    continue

                prec_c, rec_c, thresh = precision_recall_curve(y_true_cls, y_score_cls)
                f1_c = 2 * (prec_c * rec_c) / (prec_c + rec_c + 1e-6)
                best_i = np.argmax(f1_c)
                best_f1, best_conf = f1_c[best_i], thresh[min(best_i, len(thresh) - 1)]

                # ---- F1–Confidence Curve ----
                plt.figure(figsize=(4, 4))
                plt.plot(thresh, f1_c[:-1], lw=2, color="blue")
                plt.axvline(best_conf, color="red", ls="--",
                            label=f"Best F1={best_f1:.2f} @ conf={best_conf:.2f}")
                plt.title(f"{cls} — F1 vs Confidence")
                plt.xlabel("Confidence Threshold")
                plt.ylabel("F1-Score")
                plt.grid(True, ls="--", alpha=0.4)
                plt.legend()
                plt.tight_layout()
                plt.savefig(os.path.join(test_dir, f"f1_conf_curve_{cls.replace(' ', '_').lower()}.png"))
                plt.show()

                # ---- Precision–Recall Curve ----
                plt.figure(figsize=(4, 4))
                plt.plot(rec_c, prec_c, lw=2, color="purple")
                plt.title(f"{cls} — Precision–Recall Curve")
                plt.xlabel("Recall")
                plt.ylabel("Precision")
                plt.grid(True, ls="--", alpha=0.4)
                plt.tight_layout()
                plt.savefig(os.path.join(test_dir, f"pr_curve_{cls.replace(' ', '_').lower()}.png"))
                plt.show()

                # ---- Precision / Recall vs Confidence ----
                plt.figure(figsize=(4, 4))
                plt.plot(thresh, prec_c[:-1], lw=2, color="green", label="Precision")
                plt.plot(thresh, rec_c[:-1], lw=2, color="orange", label="Recall")
                plt.title(f"{cls} — Precision & Recall vs Confidence")
                plt.xlabel("Confidence Threshold")
                plt.ylabel("Score")
                plt.legend()
                plt.grid(True, ls="--", alpha=0.4)
                plt.tight_layout()
                plt.savefig(os.path.join(test_dir, f"pr_conf_curve_{cls.replace(' ', '_').lower()}.png"))
                plt.show()

            print("✅ True per-class PR/F1 curves saved successfully.")

        except Exception as e:
            print(f"⚠️ Per-class curve plotting failed: {e}")
    else:
        print("⚠️ Skipping per-class curve generation — missing data or mismatched lengths.")


def safe_visualize_results(y_true=None, y_pred=None, conf_scores=None, test_dir=None):
    """Wrapper to auto-load GT and find latest test folder."""
    if test_dir is None and 'run_dir' in globals():
        cands = glob.glob(os.path.join(run_dir, "**", "test_results"), recursive=True)
        test_dir = max(cands, key=os.path.getmtime) if cands else os.path.join(run_dir, "test_results")

    if not os.path.exists(test_dir):
        os.makedirs(test_dir, exist_ok=True)
        print(f"📁 Created missing test_results folder: {test_dir}")

    if y_true is None or len(y_true) == 0:
        labels_dir = os.path.join(dataset.location, "test", "labels")
        y_true = load_ground_truth_labels(labels_dir)

    print(f"🎨 Starting visualization for results in: {test_dir}")
    try:
        visualize_test_results(y_true, y_pred, conf_scores, test_dir)
        print("🎯 Visualization complete.")
    except Exception as e:
        print(f"❌ Visualization failed: {e}")


# 🚀 RUN VISUALIZATION
safe_visualize_results(y_true, y_pred, conf_scores, test_dir)


In [13]:
import ultralytics
ultralytics.__version__


'8.3.228'

In [ ]:
# # ===============================================================
# # # 🧩 BLOCK 6 — RELOAD & VISUALIZE SAVED RESULTS
# # ===============================================================
# # # This block allows visualization of previously saved training or test results
# # # without re-running the model inference.

# def reload_and_visualize(run_dir: str):
#     """
#     Reload saved CSVs, metrics, and images from a past YOLO run.
#     Generates inline plots (loss curves, mAP, PR, confusion matrix, etc.)
#     """
#     print(f"📂 Reloading results from: {run_dir}")
#     result_csv = os.path.join(run_dir, "results.csv")
#     summary_csv = os.path.join(run_dir, "summary_metrics.csv")
#     test_dir = os.path.join(run_dir, "test_results")

#     # 1️⃣ Summary Metrics
#     if os.path.exists(summary_csv):
#         print("\n✅ Summary Metrics (Final Epoch):")
#         df_summary = pd.read_csv(summary_csv)
#         display(df_summary.style.background_gradient(cmap="YlGnBu").format(precision=3))
#     else:
#         print("⚠️ No summary_metrics.csv found.")

#     # 2️⃣ Training Curves
#     if os.path.exists(result_csv):
#         df = pd.read_csv(result_csv)
#         print("\n📈 Training & Validation Loss Curves:")
#         plt.figure(figsize=(12, 5))
#         plt.plot(df["epoch"], df["train/box_loss"], label="Train Box Loss")
#         plt.plot(df["epoch"], df["train/cls_loss"], label="Train Class Loss")
#         if "val/box_loss" in df.columns:
#             plt.plot(df["epoch"], df["val/box_loss"], "--", label="Val Box Loss")
#         plt.xlabel("Epoch")
#         plt.ylabel("Loss")
#         plt.title("Training & Validation Loss")
#         plt.legend()
#         plt.grid(True)
#         plt.show()

#         if "metrics/mAP50(B)" in df.columns:
#             print("\n🎯 mAP Progress:")
#             plt.figure(figsize=(8, 4))
#             plt.plot(df["epoch"], df["metrics/mAP50(B)"], label="mAP@0.5")
#             plt.plot(df["epoch"], df["metrics/mAP50-95(B)"], label="mAP@0.5:0.95")
#             plt.xlabel("Epoch")
#             plt.ylabel("mAP")
#             plt.legend()
#             plt.grid(True)
#             plt.show()
#     else:
#         print("⚠️ results.csv not found in this run.")

#     # 3️⃣ Precision–Recall and Confidence Curves (auto-display if present)
#     curve_imgs = glob.glob(os.path.join(run_dir, "*curve*.png"))
#     if curve_imgs:
#         print("\n📊 Precision–Recall and Confidence Curves:")
#         for img_path in curve_imgs:
#             img = plt.imread(img_path)
#             plt.figure(figsize=(6, 4))
#             plt.imshow(img)
#             plt.axis("off")
#             plt.title(os.path.basename(img_path))
#             plt.show()
#     else:
#         print("⚠️ No PR/Confidence curve images found.")

#     # 4️⃣ Test Results (if available)
#     if os.path.exists(test_dir):
#         print("\n🧪 Test Results Summary:")
#         test_metrics = os.path.join(test_dir, "test_metrics.csv")
#         if os.path.exists(test_metrics):
#             df_test = pd.read_csv(test_metrics)
#             display(df_test.style.background_gradient(cmap="YlOrBr").format(precision=3))
#         else:
#             print("⚠️ test_metrics.csv not found in test_results.")

#         # Confusion Matrix + Per-Class Accuracy + Confidence Hist
#         img_files = glob.glob(os.path.join(test_dir, "*.png"))
#         if img_files:
#             print("\n🖼️ Saved Graphs from Test Results:")
#             for img_path in img_files:
#                 img = plt.imread(img_path)
#                 plt.figure(figsize=(6, 5))
#                 plt.imshow(img)
#                 plt.axis("off")
#                 plt.title(os.path.basename(img_path))
#                 plt.show()
#         else:
#             print("⚠️ No saved graphs found in test_results folder.")
#     else:
#         print("⚠️ No test_results directory found in this run.")
